### helper functions

In [1]:
texts = [
        "This is a sample document about machine learning and data science.",
        "Another example document discussing natural language processing techniques.",
        "More text data helps the model learn vocabulary and contextual information.",
        "This is the first document. It contains text about machine learning and data science.",
        "The second document focuses on natural language processing and machine learning applications.",
        "Text analysis is a part of data science that includes extracting keywords using TF-IDF.",
        "More documents may discuss different topics such as computer vision, AI, and deep learning."
    ]
a = " ".join(texts)
print(a)

This is a sample document about machine learning and data science. Another example document discussing natural language processing techniques. More text data helps the model learn vocabulary and contextual information. This is the first document. It contains text about machine learning and data science. The second document focuses on natural language processing and machine learning applications. Text analysis is a part of data science that includes extracting keywords using TF-IDF. More documents may discuss different topics such as computer vision, AI, and deep learning.


In [1]:
from is_economic_model.train_model import EconomicClassifier  #TODO
from is_economic_model.tdm_parser import TdmXmlParser #TODO
from sentiment_model.sentiment_score import TextAnalysis #TODO
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import joblib
from tqdm.notebook import tqdm  # Import tqdm for Jupyter Notebook
from joblib import Parallel, delayed


def modify_is_economic(dataset_file_name, parser, classifier, analyzer):
    """
    Modify an XML file by adding specific tags.

    Args:
        file_path (str): Path to the XML file to be modified.
        parser (TdmXmlParser): Parser instance to handle XML operations.
        classifier (EconomicClassifier): Classifier instance to determine economic relevance.
    returns: dataset_file_name if text is economic 
    """
    try:
        return_obg = None # initialize return object
        
        # Load the XML file
        file_path = data_path /'data_sample'/ dataset_file_name #' # get file path
        soup = parser.get_xml_soup(file_path) # get soup
        text = parser.get_art_text(soup) # get article text

        # add is_economic tag 
        is_economic_value = 1#classifier.is_economic(text) #TODO
        soup = parser.modify_tag(soup, value=is_economic_value, tag_name='is_economic') 
        if is_economic_value == 1:
            value = analyzer.text_blob_sentiment(text)
            soup = parser.modify_tag(soup, value=value, tag_name='text_blob_sentiment')
            return_obg = dataset_file_name # if article is an economic return file name
            
        # Write the modified XML back to the file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))
        
        return return_obg

    except Exception as e:
        print(f"An error occurred: {e}")
        return 


def save_file_names_to_txt(file_names, output_file):
    """
    Save a list of file names into a text file, appending to the existing content if it exists.
    
    Args:
        file_names (list): A list of file names to save.
        output_file (str): The path to the output text file.
    """
    with open(f'{output_file}.txt', 'a') as f:  # Change 'w' to 'a' to append
        for file_name in file_names:
            f.write(file_name + '\n')
      
            
def read_file_names_in_chunks(input_file, chunk_size):
    """
    Open a text file and read each row as a file name in chunks.
    
    Args:
        input_file (str): The path to the input text file.
        chunk_size (int): The number of lines (file names) to read per chunk.
        
    Yields:
        list: A list of file names read from the text file, chunk by chunk.
    """
    with open(f'{input_file}.txt', 'r') as f:
        chunk = []
        for i, line in enumerate(f, 1):
            file_name = line.strip()  # Remove any leading/trailing whitespace/newlines
            if file_name:  # Check if line is not empty
                chunk.append(file_name)
            
            if i % chunk_size == 0:
                yield chunk
                chunk = []  # Reset chunk list for the next batch

        # Yield any remaining lines in the last chunk
        if chunk:
            yield chunk


def xml_to_df(file_name, parser):
    """Read the dataset's XML file and add return into a dict.

    Args:
        dataset_path (str): Path to folder containing the dataset of articles in XML files.
        xml_file_list (list): list of the xml files containing the articles 
    Returns:
        pd.DataFrame: A DataFrame containing the concatenated data from the XML files.
    """
    # Iterate over XML files in the dataset
    file_path = f'{data_path + file_name}'
    soup = parser.get_xml_soup(file_path)
    #print(soup)
    PROPERTY_TAGS = ['GOID', 'SortTitle', 'NumericDate',
                'DocSection', 'mstar', 
                'GenSubjTerm', 'is_economic',
                'text_blob_sentiment', 'WordCount']
    
    PROPERTY_NAMES = ['GOID', 'Publisher', 'Date', 
                 'Section', 'Type', 
                 'Tags', 'is_economic',
                 'text_blob_sentiment', 'WordCount']

    content_dict = parser.get_xml_to_dict(soup, text=False, property_tags=PROPERTY_TAGS, property_names=PROPERTY_NAMES)  # get content of the file into a dict
    return content_dict


dataset_name_list = ['TheNewYorkTimes_sample20', 'USAToday_sample20', 'LosAngelesTimes_sample20', 'TheWashingtonPost_sample25']
project_path = Path('c:/Users/97253/OneDrive/Documents/work/bank of israel/financial division/yossi/tdm sentiment/tdm-sentiment/') #TODO
data_path = project_path / 'data/'


### classify articles if economic 
classify (or modify) if economic
if economic:

    - add sentiment
  
    - add economic articles names to txt file  

In [2]:
# add is_economic and text_blob_sentiment tags and write txt file with economic file names 
parser = TdmXmlParser()
classifier = EconomicClassifier()
analyzer = TextAnalysis()

chunk_size = 10000  # Adjust chunk size based on your memory and performance needs
# Split the file list into chunks

for file_chunk in read_file_names_in_chunks(data_path / 'all_dataset_file_names', chunk_size):
    with tqdm(total=len(file_chunk)) as pbar: #T
        # Use Parallel to process files in chunks
        file_names = [] #TODO
        for file_name in file_chunk: #TODO
            name = modify_is_economic(file_name, parser, classifier, analyzer) #TODO
            if name != None: #TODO
                file_names.append(name) #TODO
        #file_names = Parallel(n_jobs=-1)(delayed(modify_is_economic)(file_name, parser, classifier, analyzer) for file_name in file_chunk) 
        #file_names = [file_name for file_name in file_names if file_name]
        save_file_names_to_txt(file_names, data_path / 'economic_dataset_file_names') #T
        
        # Update the progress bar
        pbar.update(len(file_chunk))


c:\Users\97253\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BERT model 'distilbert-base-uncased-finetuned-sst-2-english' loaded successfully.


  0%|          | 0/85 [00:00<?, ?it/s]

Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob_sentiment' already exists. Updating value.
Element 'is_economic' already exists. Updating value.
Element 'text_blob

### get economic properties into df 

In [3]:
# add is_economic and text_blob_sentiment tags and write txt file with economic file names 
parser = TdmXmlParser()
classifier = EconomicClassifier()
analyzer = TextAnalysis()

chunk_size = 10000  # Adjust chunk size based on your memory and performance needs
# Split the file list into chunks
for i, file_chunk in enumerate(read_file_names_in_chunks(data_path / 'economic_dataset_file_names', chunk_size)):
    with tqdm(total=len(file_chunk)) as pbar:
        # Iterate over XML files in the dataset
        content_list = Parallel(n_jobs=-1)(delayed(xml_to_df)(file_name, parser) for file_name in file_chunk)
        df_chunk = pd.DataFrame(content_list)
        df_chunk.to_csv(f'{data_path} /economic_df_output / economic_article_df_chunk_{i}.csv', index=False)
        # Update the progress bar
        pbar.update(len(file_chunk))

BERT model 'distilbert-base-uncased-finetuned-sst-2-english' loaded successfully.


  0%|          | 0/425 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for +: 'WindowsPath' and 'str'

In [4]:
def get_all_csv_files(folder_path):
    # Use pathlib to find all CSV files in the folder
    csv_files = list(Path(folder_path).glob('*.csv'))
    return csv_files


def concat_csv_files(folder_path):
    # Get a list of all CSV files in the folder
    csv_files = get_all_csv_files(folder_path)
    # Initialize an empty list to store DataFrames
    df_list = []
    
    # Loop through the list of CSV files
    for file in csv_files:
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file)
        # Append the DataFrame to the list
        df_list.append(df)
    
    # Concatenate all DataFrames in the list into a single DataFrame
    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

# Folder paths
folder_path = data_path / 'economic_df_output'

# Combine CSV files from both folders
combined_df = concat_csv_files(folder_path)

# Drop duplicate rows based on the 'GOID' column
combined_df = combined_df.drop_duplicates(subset=['GOID'])

# Combine 'International Herald Tribune' and 'New York Times' into one category
combined_df['Publisher'] = combined_df['Publisher'].replace(
    {'international herald tribune': 'new york times the',
     'new york times the': 'new york times the'}
)

# The combined_df now contains the data from both folders with duplicates removed and publishers combined
combined_df


,GOID,Publisher,Date,Section,Type,Tags,is_economic,text_blob_sentiment
0,1034336215,new york times,2012-08-21,a,news,stop & frisk,1,0.048857
1,1531107677,new york times,2014-05-18,sp,news,NaN,1,0.000000
2,1620547810,new york times,2014-11-06,d,news,furniture,1,0.235714
3,1625069727,new york times,2014-11-15,a,news,battleships,1,0.030058
4,1690306948,new york times,2015-06-23,a,news,budgets,1,0.072455
...,...,...,...,...,...,...,...,...
80,742633964,washington post the,2010-08-13,a section,news,gays & lesbians,1,0.036379
81,755900909,washington post the,2010-10-01,a section,commentary,renovation & restoration,1,0.082393
82,893725652,washington post the,2011-09-22,local living,feature,herbicides,1,0.061500
83,894525606,washington post the,2011-09-29,style,general information,NaN,1,0.109917


In [ ]:
with open(data_path / 'all_dataset_file_names.txt', 'r') as f:
    file_name = iter(f)
    file_name = next(file_name).strip()
 
#print(file_name)   
    
project_path = Path('c:/Users/97253/OneDrive/Documents/work/bank of israel/financial division/yossi/tdm sentiment/tdm-sentiment/') #TODO
data_path = project_path / 'data/'

path = data_path / 'data_sample' / file_name
#print(path)

parser = TdmXmlParser()
soup = parser.get_xml_soup(path)
text1 = parser.get_art_text(soup)

import re
#for text in text1.split('.'):
    #text = re.sub(r'\(photograph by [^\)]+\)', '', text)
    # Remove URLs and any other unwanted patterns
    #text = re.sub(r'http\S+', '', text)  # Remove URLs
    #text = re.sub(r'[^a-z0-9\s]', '', text)  # Keep only letters, numbers, and spaces
        

print(text1)

In [ ]:
parser = TdmXmlParser()
classifier = EconomicClassifier()
analyzer = TextAnalysis()

a = analyzer.bert_sentiment(text1)
a

In [5]:
import pandas as pd

# Sample DataFrame
data = {
    'WordCount': [500, 750, 600, 800, 650, 700],
    'Date': ['2021-01-01', '2021-01-01', '2021-01-01',
             '2021-01-02', '2021-01-02', '2021-01-02'],
    'page': [1, 1, 2, 1, 2, 2]
}

df = pd.DataFrame(data)
df

,WordCount,Date,page
0,500,2021-01-01,1
1,750,2021-01-01,1
2,600,2021-01-01,2
3,800,2021-01-02,1
4,650,2021-01-02,2
5,700,2021-01-02,2


In [8]:

# Step 1: Sum of words for each page on each day
page_word_sums = df.groupby(['Date', 'page'])['WordCount'].sum().reset_index()
print("Sum of words for each page on each day:")
print(page_word_sums)

# Step 2: Average words per page for each day
average_words_per_page = page_word_sums.groupby('page')['WordCount'].mean().reset_index()
average_words_per_page.rename(columns={'WordCount': 'AverageWordsPerPage'}, inplace=True)
print("\nAverage words per page for each day:")
print(average_words_per_page)


Sum of words for each page on each day:
         Date  page  WordCount
0  2021-01-01     1       1250
1  2021-01-01     2        600
2  2021-01-02     1        800
3  2021-01-02     2       1350

Average words per page for each day:
   page  AverageWordsPerPage
0     1               1025.0
1     2                975.0


In [43]:
import pandas as pd
import json
from pathlib import Path

# Sample DataFrame for a single newspaper
data = {
    'WordCount': [500, 750, 600, 800, 650, 700],
    'Date': ['2021-01-01', '2021-01-01', '2021-01-01',
             '2021-01-02', '2021-01-02', '2021-01-02'],
    'page': [1, 1, 2, 1, 2, 2],
    'Newspaper': ['Times'] * 6  # Only one newspaper
}

df = pd.DataFrame(data)

# Ensure that the 'Date' column is of datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Get the newspaper name
newspaper_name = df['Newspaper'].iloc[0]

# Step 1: Sum of words for each page and date
page_word_sums = df.groupby(['Date', 'page'])['WordCount'].sum().reset_index()

# Step 2: Calculate average words per page
average_words_per_page = page_word_sums.groupby('page')['WordCount'].mean().reset_index()
average_words_per_page.rename(columns={'WordCount': 'AverageWordsPerPage'}, inplace=True)

# Convert page numbers to strings for JSON keys
page_size_dict = average_words_per_page.set_index('page')['AverageWordsPerPage'].to_dict()
page_size_dict = {str(key): value for key, value in page_size_dict.items()}

# Store the dictionary using JSON
storage_dir = Path('newspaper_page_sizes')
storage_dir.mkdir(parents=True, exist_ok=True)
file_path = storage_dir / f"{newspaper_name}_page_sizes.json"

with file_path.open('w', encoding='utf-8') as f:
    json.dump(page_size_dict, f, ensure_ascii=False, indent=4)
#print(f"Stored page sizes for {newspaper_name} in {file_path}")
df

,WordCount,Date,page,Newspaper
0,500,2021-01-01,1,Times
1,750,2021-01-01,1,Times
2,600,2021-01-01,2,Times
3,800,2021-01-02,1,Times
4,650,2021-01-02,2,Times
5,700,2021-01-02,2,Times


In [44]:
df = pd.read_csv(output_file)

# Ensure that the 'Date' column is of datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Function to normalize page identifiers
def normalize_page(page):
    parts = page.split('.')
    if len(parts) == 2:
        # Sort parts to ensure consistent ordering
        return '.'.join(sorted(parts))
    return page

# Apply normalization
df['Page'] = df['Page'].apply(normalize_page)

# Step 1: Sum of words for each page and date
page_word_sums = df.groupby(['Date', 'Page'])['WordCount'].sum().reset_index()

# Step 2: Calculate average words per page
average_words_per_page = page_word_sums.groupby('Page')['WordCount'].mean().reset_index()
average_words_per_page.rename(columns={'WordCount': 'AverageWordsPerPage'}, inplace=True)

# Convert page numbers to strings for JSON keys
page_size_dict = average_words_per_page.set_index('Page')['AverageWordsPerPage'].to_dict()
page_size_dict = {str(key): value for key, value in page_size_dict.items()}

#storage_dir.mkdir(parents=True, exist_ok=True)
file_path = data_path / f"newspaper_page_sizes/{data_set}.json" #TODO

with file_path.open('w', encoding='utf-8') as f:
    json.dump(page_size_dict, f, ensure_ascii=False, indent=4)
print(f"Stored page sizes for {newspaper_name} in {file_path}")
page_size_dict

,page,AverageWordsPerPage
0,1,1250
1,2,1350


In [ ]:
def get_page_size_cas(dataset_name, page_tag, data_path):
    """
    Retrieves the page size from the JSON file based on newspaper and page_tag.
    
    Args:
        newspaper (str): Name of the newspaper.
        page_tag (str): Normalized page identifier.
        storage_dir (str): Directory where JSON files are stored.
    
    Returns:
        int: Page size or None if not found.
    """
    file_path = data_path / f"newspaper_page_sizes/{dataset_name}.json"
    if not file_path.exists():
        print(f"Page sizes file not found for newspaper: {dataset_name}")
        return None
    
    with file_path.open('r', encoding='utf-8') as f:
        page_data = json.load(f)
    
    page_info = page_data.get(page_tag)
    if page_info:
        return page_info.get('AverageWordsPerPage', None)
    else:
        print(f"Page '{page_tag}' not found in {file_path}")
        return None
